In [13]:
from Adafruit_IO import Client, Feed, Group
from i2c_sensor_classes import Ky015Sensor, Ky018Sensor
from aio_helper_classes import HabitatMonitorClient
import time

In [14]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

In [122]:
class HabitatMonitorServer(Client):
    def __init__(self, aio_uname, aio_key):
        super().__init__(aio_uname, aio_key)
        print('Creating Habitat MOn Server...')
        self.client_ids = self.get_client_ids()
        self.high_bright_feed = self.feeds('high-brightness-threshold')
        self.high_hum_feed= self.feeds('high-humidity-threshold')
        self.high_temp_feed = self.feeds('high-temperature-threshold')
        self.low_bright_feed = self.feeds('low-brightness-threshold')
        self.low_hum_feed = self.feeds('low-humidity-threshold')
        self.low_temp_feed = self.feeds('low-temperature-threshold')
        self.temperature_group = self.groups('temperature-measurements')
        self.humidity_group = self.groups('humidity-measurements')
        self.brightness_group = self.groups('brightness-measurements')
        self.high_temp, self.low_temp = None, None
        self.high_hum, self.low_hum = None, None
        self.high_bright, self.low_bright = None, None
        
    def get_client_ids(self):
        client_id_list = []
        id_feed = self.feeds('client-ids')
        cids = self.data(id_feed.key)
        for client_id in cids:
            print(client_id.value)
            client_id_list.append(client_id.value)
        return client_id_list
    
    @property
    def high_temp(self):
        return self.__high_temp
    @high_temp.setter
    def high_temp(self, val):
        if val != None:
            self.send(self.high_temp_feed.key, val)
        self.__high_temp = val
    @property
    def high_hum(self):
        return self.__high_hum
    @high_hum.setter
    def high_hum(self, val):
        if val != None:
            self.send(self.high_hum_feed.key, val)
        self.__high_hum = val
    @property
    def high_bright(self):
        return self.__high_bright
    @high_bright.setter
    def high_bright(self, val):
        if val != None:
            self.send(self.high_bright_feed.key, val)
        self.__high_bright = val
    @property
    def low_temp(self):
        return self.__low_temp
    @low_temp.setter
    def low_temp(self, val):
        if val != None:
            self.send(self.low_temp_feed.key, val)
        self.__low_temp = val
    @property
    def low_hum(self):
        return self.__low_hum
    @low_hum.setter
    def low_hum(self, val):
        if val != None:
            self.send(self.low_hum_feed.key, val)
        self.__low_hum = val
    @property
    def low_bright(self):
        return self.__low_bright
    @low_bright.setter
    def low_bright(self, val):
        if val != None:
            self.send(self.low_bright_feed.key, val)
        self.__low_bright = val       

# Server Code

In [123]:
def server_process(uname, key):
    #create an aio client 
    aio = HabitatMonitorServer(uname, key)
    #get the feeds for setting thresholds
    aio.high_temp, aio.low_temp = 67, 11
    aio.high_hum, aio.low_hum = 64, 27
    aio.high_bright, aio.low_bright = 1205, 410
    monitor_thresholds(aio)
    
def monitor_thresholds(aio:HabitatMonitorServer):
    temp_is_low, temp_is_high = monitor_temp_thresholds(aio)
    hum_is_high, hum_is_low = monitor_hum_thresholds(aio)
    bright_is_low, bright_is_high = monitor_bright_thresholds(aio)
    
def monitor_temp_thresholds(aio:HabitatMonitorServer):
    for tfeed in aio.temperature_group.feeds:
        last_reading = float(aio.receive(tfeed.key).value)
        if last_reading > aio.high_temp:
            print('Warning: Temp={} > {}!'.format(last_reading, aio.high_temp))
            return False, True
        elif last_reading < aio.low_temp():
            print('Warning: Temp={} > {}!'.format(last_reading, aio.high_temp))
            return True, False
        else:
            print('All good {} < Temp={} < {}'.format(aio.low_temp, last_reading, aio.high_temp))
            return 

def monitor_hum_thresholds(aio:HabitatMonitorServer):
    for hfeed in aio.humidity_group.feeds:
        print(hfeed.key)
        last_reading = aio.receive(hfeed.key).value
        if last_reading > aio.high_hum:
            print('Warning: Temp={} > {}!'.format(last_reading, aio.high_hum))
            return False, True
        elif last_reading < aio.low_hum():
            print('Warning: Temp={} > {}!'.format(last_reading, aio.high_hum))
            return True, False
        else:
            print('All good {} < Temp={} < {}'.format(aio.low_hum, last_reading, aio.high_hum))
            return 
        
def monitor_bright_thresholds(aio:HabitatMonitorServer):
    for bfeed in aio.humidity_group.feeds:
        print(bfeed.key)
        last_reading = aio.receive(bfeed.key).value
        if last_reading > aio.high_bright:
            print('Warning: Temp={} > {}!'.format(last_reading, aio.high_bright))
            return False, True
        elif last_reading < aio.low_hum():
            print('Warning: Temp={} > {}!'.format(last_reading, aio.high_bright))
            return True, False
        else:
            print('All good {} < Temp={} < {}'.format(aio.low_bright, last_reading, aio.high_bright))
            return 

In [124]:
uname, key = 'vsaah', 'aio_frxg67OsqvjWMcJorgxHhOtdev7M'
server_process(uname, key)

Creating Habitat MOn Server...
ryans-pynq-board
joeys-pynq-board


TypeError: 'int' object is not callable